In [1]:
import sys
sys.path.append("/Users/jarvixwang/Documents/Project/gpt-researcher")
import json
from agent import prompts
from agent import llm_utils #import create_chat_completion

model_names = {"gpt4": "gpt-4",
               "gpt-3.5": "gpt-35-turbo"}

selenium_web_browser: chrome


In [2]:
prompts.generate_search_queries_prompt(question="large language models")

'Write 3 google search queries to search online that form an objective opinion from the following: "large language models".\nUse the current date if needed: October 16, 2023.\nYou must respond with a list of strings in the following format: ["query 1", "query 2", "query 3"].'

### 以下是为了测试[llm_utils.choose_agent(task)](https://github.com/lan2720/gpt-researcher/blob/5c55192c05fe834a0c52ffcf585446373bc2e3bb/agent/llm_utils.py#L101)的功能
- 生成agent_name和agent_role_prompt

In [3]:
agent_role_prompt = prompts.auto_agent_instructions()
question = "generate 3 names for girl"
action = f"task: {question}"
messages = [{
            "role": "system",
            "content": agent_role_prompt
        }, {
            "role": "user",
            "content": action,
        }]

rsp = llm_utils.create_chat_completion(messages=messages, model=model_names["gpt-3.5"], temperature=0)
data = json.loads(rsp.strip())
for k, v in data.items():
    print(f"{k}: {v}")

agent: 👶 Baby Name Agent
agent_role_prompt: You are an AI assistant specialized in baby names. Your goal is to provide unique, meaningful, and popular names for babies based on various criteria.


### [ResearchAgent](https://github.com/lan2720/gpt-researcher/blob/5c55192c05fe834a0c52ffcf585446373bc2e3bb/agent/research_agent.py#L25)做科研, 就2步
- conduct_research(): 得到self.research_summary
    - [create_search_queries()](https://github.com/lan2720/gpt-researcher/blob/5c55192c05fe834a0c52ffcf585446373bc2e3bb/agent/research_agent.py#L93): 根据当前task生成多个搜索问题, 是GPT生成的
        - 对每个saerch_query生成该query的summary: [run_search_summary(query)](https://github.com/lan2720/gpt-researcher/blob/5c55192c05fe834a0c52ffcf585446373bc2e3bb/agent/research_agent.py#L120)
            - responses = await self.[async_search(query)](https://github.com/lan2720/gpt-researcher/blob/5c55192c05fe834a0c52ffcf585446373bc2e3bb/agent/research_agent.py#L102): DDGS(), duckduckgo搜索
                - 针对这个query进行搜索: [web_search(query)](https://github.com/lan2720/gpt-researcher/blob/5c55192c05fe834a0c52ffcf585446373bc2e3bb/actions/web_search.py#L7)
                - 获取所有返回页面的urls: self.[get_new_urls([url.get("href") for url in search_results])](https://github.com/lan2720/gpt-researcher/blob/5c55192c05fe834a0c52ffcf585446373bc2e3bb/agent/research_agent.py#L61)
                - 浏览每个网页: [async_browse(url, query, self.websocket)](https://github.com/lan2720/gpt-researcher/blob/5c55192c05fe834a0c52ffcf585446373bc2e3bb/actions/web_scrape.py#L36), 实际[async_browse()实现](https://github.com/lan2720/gpt-researcher/blob/5c55192c05fe834a0c52ffcf585446373bc2e3bb/actions/web_scrape.py#L36)
                    - 获取当前url页面的文本: driver, text = await loop.run_in_executor(executor, scrape_text_with_selenium, url)
                    - 给浏览器driver增加header: await loop.run_in_executor(executor, add_header, driver)
                    - 对文本总结: [summary_text](https://github.com/lan2720/gpt-researcher/blob/5c55192c05fe834a0c52ffcf585446373bc2e3bb/actions/web_scrape.py#L57C9-L57C21) = await loop.run_in_executor(executor, summary.summarize_text, url, text, question, driver), 真正的[summary_text实现在processing/text.py内](https://github.com/lan2720/gpt-researcher/blob/5c55192c05fe834a0c52ffcf585446373bc2e3bb/processing/text.py#L46)
                    - 返回str: f"Information gathered from url {url}: {summary_text}"
        - 全部拼接起来作为self.research_summary
- 根据report_type的不同，写报告: write_report(report_type, websocket)

### 如何产生websocket?
- [code](https://github.com/lan2720/gpt-researcher/blob/5c55192c05fe834a0c52ffcf585446373bc2e3bb/main.py#L39C1-L41C37)
```
manager = WebSocketManager()

@app.websocket("/ws")
async def websocket_endpoint(websocket: WebSocket):
    await manager.connect(websocket)
```